In [1]:
!pip install keras_tuner 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 1.4 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import files 
import keras_tuner as kt

In [3]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Copied from AlphabetSoupCharity 
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN','NAME'], inplace = True)

# Bin application types, low counts into Other
application_types_to_replace = ['T17','T15','T29','T14','T25','T2','T12','T13','T9']

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Bin classifications 
classifications_to_keep = ['C1000','C2000','C1200','C3000','C2100']

for index, row in application_df.iterrows():
  if row['CLASSIFICATION'] not in classifications_to_keep:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(row['CLASSIFICATION'],"Other")

# Convert 'SPECIAL_CONSIDERATIONS' from Y/N to 1 and 0 in a single column (don't rerun this, columns will be NaN)
application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].map({'Y' : int(1), 'N' : int(0)})

# Convert categorical data to numeric with `pd.get_dummies`
application_encoded_df = pd.get_dummies(application_df, columns = ['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT'])
application_encoded_df.head()

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,5000,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,108590,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,5000,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,1,0,6692,1,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,0,142590,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0


In [5]:
# Split our preprocessed data into our features and target arrays
y = application_encoded_df.IS_SUCCESSFUL.values
X = application_encoded_df.drop(columns = 'IS_SUCCESSFUL').values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:


# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# Defining hidden layers
nn.add(tf.keras.layers.Dense(units=46, activation="relu", input_dim=42))
nn.add(tf.keras.layers.Dense(units=6, activation="relu"))
nn.add(tf.keras.layers.Dense(units=26, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 46)                1978      
                                                                 
 dense_1 (Dense)             (None, 6)                 282       
                                                                 
 dense_2 (Dense)             (None, 26)                182       
                                                                 
 dense_3 (Dense)             (None, 1)                 27        
                                                                 
Total params: 2,469
Trainable params: 2,469
Non-trainable params: 0
_________________________________________________________________


# Testing Top 3 Models suggesting by the Keras Tuner 

In [8]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 4s 3ms/step - loss: 0.5769 - accuracy: 0.7154
Epoch 2/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5531 - accuracy: 0.7312
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7322
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7333
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7338
Epoch 6/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5474 - accuracy: 0.7338
Epoch 7/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5471 - accuracy: 0.7326
Epoch 8/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5471 - accuracy: 0.7338
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7341
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7350

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5555 - accuracy: 0.7235 - 830ms/epoch - 3ms/step
Loss: 0.5554735064506531, Accuracy: 0.723498523235321


In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn2 = tf.keras.models.Sequential()

# Defining hidden layers
nn2.add(tf.keras.layers.Dense(units=11, activation="tanh", input_dim=42))
nn2.add(tf.keras.layers.Dense(units=6, activation="tanh"))
nn2.add(tf.keras.layers.Dense(units=16, activation="tanh"))
nn2.add(tf.keras.layers.Dense(units=26, activation="tanh"))
nn2.add(tf.keras.layers.Dense(units=6, activation="tanh"))
nn2.add(tf.keras.layers.Dense(units=21, activation="tanh"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 11)                473       
                                                                 
 dense_9 (Dense)             (None, 6)                 72        
                                                                 
 dense_10 (Dense)            (None, 16)                112       
                                                                 
 dense_11 (Dense)            (None, 26)                442       
                                                                 
 dense_12 (Dense)            (None, 6)                 162       
                                                                 
 dense_13 (Dense)            (None, 21)                147       
                                                                 
 dense_14 (Dense)            (None, 1)                

In [15]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
fit_model2 = nn2.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 4s 2ms/step - loss: 0.5848 - accuracy: 0.7163
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5649 - accuracy: 0.7286
Epoch 3/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5585 - accuracy: 0.7310
Epoch 4/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5560 - accuracy: 0.7313
Epoch 5/20
804/804 [==============================] - 5s 6ms/step - loss: 0.5538 - accuracy: 0.7324
Epoch 6/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5522 - accuracy: 0.7334
Epoch 7/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5523 - accuracy: 0.7320
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7337
Epoch 9/20
804/804 [==============================] - 7s 8ms/step - loss: 0.5505 - accuracy: 0.7336
Epoch 10/20
804/804 [==============================] - 10s 12ms/step - loss: 0.5504 - accuracy: 0.73

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn3 = tf.keras.models.Sequential()

# Defining hidden layers
nn3.add(tf.keras.layers.Dense(units=41, activation="sigmoid", input_dim=42))
nn3.add(tf.keras.layers.Dense(units=26, activation="sigmoid"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 41)                1763      
                                                                 
 dense_16 (Dense)            (None, 26)                1092      
                                                                 
 dense_17 (Dense)            (None, 1)                 27        
                                                                 
Total params: 2,882
Trainable params: 2,882
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
fit_model3 = nn2.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5465 - accuracy: 0.7335
Epoch 2/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5459 - accuracy: 0.7362
Epoch 3/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5458 - accuracy: 0.7350
Epoch 4/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5454 - accuracy: 0.7355
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7363
Epoch 6/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5454 - accuracy: 0.7370
Epoch 7/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5452 - accuracy: 0.7364
Epoch 8/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5449 - accuracy: 0.7353
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7353
Epoch 10/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5447 - accuracy: 0.7365

In [20]:
# Export our model to HDF5 file
nn3.save('AlphabetSoupCharity_Optimization.h5')

# This code is run in Google colab, so need to download it manually 
files.download('AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>